<a href="https://colab.research.google.com/github/Legendary-Nerd/Mental-Health-RAG-Chatbot/blob/main/Health_rag_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG (Retrieval-Augmented Generation) App

Upload a PDF, ask questions about its content.

In [ ]:
# Install required packages (uncomment if needed)
!pip install pypdf faiss-cpu sentence-transformers google-generativeai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 970.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [ ]:
import os
import faiss
import numpy as np
import google.generativeai as genai
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
load_dotenv()
from pathlib import Path

## 1. Set PDF Path

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("elvis23/mental-health-conversational-data")

print("Path to dataset files:", path)

100%|██████████| 11.8k/11.8k [00:00<00:00, 5.33MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/elvis23/mental-health-conversational-data/versions/1


In [ ]:
import os
os.listdir(path)

['intents.json']

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import json
from pathlib import Path

# Set the path to your JSON file
json_path = path +'/intents.json'  # Update with your actual file path
json_file = Path(json_path).expanduser()
print(f"Using JSON file: {json_file}")

# Load the JSON content

try:
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
except Exception as e:
    print(f"Error reading JSON: {e}")
    data = {}



Using JSON file: /root/.cache/kagglehub/datasets/elvis23/mental-health-conversational-data/versions/1/intents.json


In [ ]:
# Set the path to your PDF file
# pdf_path = '/content/CST401 M5 Ktunotes.in.pdf' # Provide path properly like: ./report.pdf, C:\\Users\\YOUR_USERNAME\\path\\project.pdf
# pdf_file = Path(pdf_path).expanduser()
# print(f"Using PDF file: {pdf_file}")

Using PDF file: /content/CST401 M5 Ktunotes.in.pdf


## 2. Extract Text from PDF

In [ ]:
# # prompt: read multiple pdf files and crate chunks

# def read_pdfs(directory_path):
#     """Reads all PDF files in a directory and extracts text."""
#     all_text = ""
#     for filename in os.listdir(directory_path):
#         if filename.endswith(".pdf"):
#             file_path = os.path.join(directory_path, filename)
#             try:
#                 reader = PdfReader(file_path)
#                 for page_num in range(len(reader.pages)):
#                     all_text += reader.pages[page_num].extract_text() + "\n"
#             except Exception as e:
#                 print(f"Error reading {filename}: {e}")
#     return all_text

# # Example usage:
# directory_of_pdfs = '/content/my_pdfs/' # Replace with the path to your directory containing PDFs
# text_from_pdfs = read_pdfs(directory_of_pdfs)
# text_chunks = chunk_text(text_from_pdfs)
# print(f"Read {len(text_from_pdfs.split())} words from PDFs.")
# print(f"Created {len(text_chunks)} chunks.")


In [ ]:
all_text = ""
if "intents" in data:
    for intent in data["intents"]:
        all_text += " ".join(intent.get("patterns", [])) + "\n"
else:
    print("No intents found.")

print(f"Extracted {len(all_text)} characters from JSON.")

Extracted 5437 characters from JSON.


## 3. Chunk Text

In [ ]:
def chunk_text(text, chunk_size=500, overlap=100):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+chunk_size]
        chunks.append(' '.join(chunk))
        i += chunk_size - overlap
    return chunks

chunks = chunk_text(all_text)
print(f"Total chunks: {len(chunks)}")

Total chunks: 3


## 4. Embed Chunks and Build Vector Store

In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(chunks, show_progress_bar=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))

print(f"FAISS index built with {index.ntotal} vectors.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

FAISS index built with 3 vectors.


## 5. Ask Questions

In [ ]:
genai.configure(api_key="###")

# Initialize Gemini model
model = genai.GenerativeModel('gemini-2.0-flash')

def retrieve(query, k=3):
    q_emb = embedder.encode([query])
    D, I = index.search(np.array(q_emb).astype('float32'), k)
    return [chunks[i] for i in I[0]]

def rag_answer(query):
    context = '\n'.join(retrieve(query))
    prompt = f"""Use the following context to answer the question. Keep your answer concise and relevant.

    Context:
    {context}

    Question: {query}
    Answer:"""

    response = model.generate_content(prompt)
    return response.text.strip()

## 6. Interactive Chat

In [ ]:
print("Chat with your PDF! Type 'exit' to quit.\n")
while True:
    query = input("You: ")
    if query.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break
    answer = rag_answer(query)
    print(f"Bot: {answer}\n")


Chat with your PDF! Type 'exit' to quit.

You: hi
Bot: Hi Hey Is anyone there?

You: i feel sick
Bot: I am sorry to hear that you are feeling sick. I am not a medical professional and cannot provide medical advice. If you are concerned about your health, please consult a doctor or other qualified healthcare provider.

You: I want to talk about something else
Bot: Okay. What would you like to talk about?

You: is benny cool
Bot: This question is not related to the context. Therefore, I cannot answer it.

You: exit
Goodbye!
